# Packages

In [1]:
from quocslib.optimalcontrolproblems.OneQubitProblem_2fields import OneQubit2Fields
import time

import scipy
import numpy as np
from scipy import linalg
import os
import sys
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.optimize import minimize
import time
import copy
from scipy import signal
# import necessary stuff from quidi
from logic.pulsed.pulse_objects import PulseBlock, PulseBlockEnsemble, PulseSequence

# QuOCS and Qudi with Noise

## Creation of the optimization dictionary

The optimization dictionary contains all the settings compulsory for the optimization algorithm in order to run a proper optimization.

In [2]:
optimization_client_name = "test_dCRAB_Noisy_2_control_fields"

optimization_dictionary = {"optimization_client_name": optimization_client_name,
                           'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABNoisyAlgorithm', 
                           'opti_algorithm_class': 'DCrabNoisyAlgorithm', 
                          }

Number of iteration and super-iterations

In [3]:
# Total number of dCRAB superiteration
super_iteration_number = 10000000
# Maximum number of iteration per super-iteration (in max_eval_total: maximale anzahl von evaluation steps gesamt)
maximum_function_evaluations_number = 500

To activate the drift compensation and the re-evaluation steps add

In [4]:
optimization_dictionary['algorithm_settings'] = {"algorithm_name": "dCRAB",
                                                 "super_iteration_number": super_iteration_number,
                                                 #"max_eval_total": super_iteration_number * maximum_function_evaluations_number,
                                                 "FoM_goal": 0.00001,
                                                 "total_time_lim": 3*60,
                                                 "compensate_drift": {
                                                     "compensate_after_SI": True,
                                                     "compensate_after_minutes": 15
                                                 },
                                                 "random_number_generator":{
                                                     "seed_number":420
                                                 },
                                                 #"re_evaluation": {
                                                     #"re_evaluation_steps": [0.3, 0.5, 0.51]
                                                 #}
}

Settings for the inner algorithm used by dCRAB

In [5]:
optimization_dictionary['algorithm_settings']["dsm_settings"] = {'general_settings': {
                                                "dsm_algorithm_name": "NelderMead", 
                                                'is_adaptive': False
                                            }, 
                                            'stopping_criteria': {
                                                #"max_eval": 100,
                                                #"time_lim": 45,
                                                #"xatol": 1e-10, 
                                                #"frtol": 1e-10,
                                                "change_based_stop": {
                                                    "cbs_funct_evals": 100,
                                                    "cbs_change": 0.001
                                                }
                                            }
}

### Times

In [6]:
time_p = {'time_name': 'time_p', 'initial_value': 201e-9} # pulse length
optimization_dictionary["times"] = [time_p]

### Parameters

In [7]:
optimization_dictionary['parameters'] = []

### Pulses

In [8]:
vec_number = 2
upper_freq_lim = 8

pulse_amplitude = {'pulse_name': 'Amplitude', 
                           'upper_limit': 0.5, 
                           'lower_limit': -0.5, 
                           'bins_number': 1001, 
                           'time_name': 'time_p', 
                           'amplitude_variation': 0.1, # added to guess amp, to create 3 points for start simplex
                           'basis': {'basis_name': 'Fourier', 
                                     'basis_class': 'Fourier', 
                                     'basis_module': 'quocslib.pulses.basis.Fourier', 
                                     'basis_vector_number': vec_number, # number of frequencies within the below defined range
                                     'random_super_parameter_distribution': 
                                     {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                                           'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                                           'lower_limit': 0.0, 'upper_limit': upper_freq_lim} # number of oscillations within pulse
                                    }, 
                           'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
                           'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.03245 + 0.0*t'}
                          }

pulse_phase = {'pulse_name': 'Phase', 
               'upper_limit': 0.5, 
               'lower_limit': -0.5, 
               'bins_number': 1001, 
               'time_name': 'time_p', 
               'amplitude_variation': 0.1, # added to guess amp, to create 3 points for start simplex
               'basis': {'basis_name': 'Fourier', 
                         'basis_class': 'Fourier', 
                         'basis_module': 'quocslib.pulses.basis.Fourier', 
                         'basis_vector_number': vec_number, # number of frequencies within the below defined range
                         'random_super_parameter_distribution': 
                         {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                               'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                               'lower_limit': 0.0, 'upper_limit': upper_freq_lim} # number of oscillations within pulse
                        }, 
               'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
               'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.0*t'}
              }

optimization_dictionary['pulses'] = [pulse_amplitude, pulse_phase]

### Put all together and get ready to start the optimization with Qudi-QuOCS

In [11]:
opti_comm_dict = {"optimization_dictionary": optimization_dictionary}
optimizationlogic.load_opti_comm_dict(opti_comm_dict)

### Important: If the GUI is not showing the optimization dictionary, restart the Kernel

Print the optimization dictionary also here

In [12]:
optimalcontrol.opti_comm_dict

{'optimization_dictionary': {'optimization_client_name': 'test_dCRAB_Noisy_2_control_fields', 'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABNoisyAlgorithm', 'opti_algorithm_class': 'DCrabNoisyAlgorithm', 'algorithm_settings': {'algorithm_name': 'dCRAB', 'super_iteration_number': 10000000, 'FoM_goal': 1e-05, 'total_time_lim': 180, 'compensate_drift': {'compensate_after_SI': True, 'compensate_after_minutes': 15}, 'random_number_generator': {'seed_number': 420}, 'dsm_settings': {'general_settings': {'dsm_algorithm_name': 'NelderMead', 'is_adaptive': False}, 'stopping_criteria': {'change_based_stop': {'cbs_funct_evals': 100, 'cbs_change': 0.001}}}}, 'times': [{'time_name': 'time_p', 'initial_value': 2.01e-07}], 'parameters': [], 'pulses': [{'pulse_name': 'Amplitude', 'upper_limit': 0.5, 'lower_limit': -0.5, 'bins_number': 1001, 'time_name': 'time_p', 'amplitude_variation': 0.1, 'basis': {'basis_name': 'Fourier', 'basis_class': 'Fourier', 'basis_module': 'quocslib.pulses.basis.F

# Measurement

In [ ]:
######################################################################################################
# Parameters and Settings
######################################################################################################

# file paths 
folder_path = r'C:\Users\Mesoscopic\Desktop\quocs_pulses'
filename_amplitude=r'amplitude'
filename_phase=r'phase'

#FoM_path= r'C:\Users\Mesoscopic\Desktop\Redcrab_data\FoM'

# runtime of the experiment
runtime = 5

# needs to be smaller than the actual pulse length time_p due to the interpolation
length_oc = 200e-9

# parameter to stop the experiment if its set to False in the console
pulsedmasterlogic.globalrun = True

# Dictonary containing the general measurement parameters for the predefined measurements
globaldict = pulsedmasterlogic.generation_parameters
globaldict["laser_channel"] = "d_ch1" 
globaldict["sync_channel"] = "d_ch2" 
globaldict["gate_channel"] = "d_ch1" 
globaldict["microwave_channel"] = "a_ch1"
globaldict["microwave_amplitude"] = 0.5
globaldict["microwave_frequency"] = 1.281720e9
globaldict["rabi_period"] = 56.51e-9
globaldict["laser_length"] = 25e-06
globaldict["laser_delay"] = 0
globaldict["wait_time"] = 2e-06
globaldict["analog_trigger_voltage"] = 0.0

# load the general measurement parameters for the predefined measurements
pulsedmasterlogic.set_generation_parameters(globaldict)

# tell the measurement gui how the sequence looks
pulsed_settings = dict()
pulsed_settings['invoke_settings'] = True
pulsedmasterlogic.set_measurement_settings(pulsed_settings)

# make sure everything is finished
time.sleep(5)

# how the laser response is analyzed    
pulsedmasterlogic.set_analysis_settings(method = 'mean')
pulsedmasterlogic.set_analysis_settings(signal_start = 150e-9)
pulsedmasterlogic.set_analysis_settings(signal_end = 1e-6)
pulsedmasterlogic.set_extraction_settings(method = 'conv_deriv')

# array to save the fom evolution
fom_all = []

# This section is devoted to the initialization in the pulsed logic and optimization logic of the main
# settings and parameters to be usde in the creation ofthe pulse sequence and the optimization
# Iteration, controls and figure of merit to compare with QuOCS
# Just an example for debug
args_dict = {"is_noisy": True}
qubit = OneQubit2Fields(args_dict)

######################################################################################################
# Measurement
######################################################################################################
optimalcontrol.start_optimization()

# crucial, maybe it can be much shorter
time.sleep(1)

# Just a time to check for latent time
last_time_fom = time.time()
# repeat the whole process until its manually stopped or QuOCS finsihed the optimization
# Wait few seconds before starting to get and return data
while not optimizationlogic.handle_exit_obj.is_user_running:
    time.sleep(0.1)
    if (time.time() - last_time_fom) > 30:
        print("Problem at the beginning! Surpassed the 30 secs")
        break

# iteration number
it_val = 0

# when did the optimization start?
opt_start_time = time.time()
    
# print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
while  optimizationlogic.handle_exit_obj.is_user_running == True:
    time_stamp=time.time()
    # wait until QuOCS optimizes the controls
    # print("Wait until the controls logic gives the controls")
    while not controlslogic.are_pulses_calculated:
        time.sleep(0.1)
        # If the waiting time exceed 10 seconds left stop the optimization
        if time.time() - last_time_fom > 20:
            print("Too much time... Exit!")
            optimizationlogic.handle_exit_obj.is_user_running = False
            break
            
    # If time exceeded, exit from the while loop and give a very high fom to the optimization algorithm
    # and status code -1, to interrupt the optimization smoothly
    if optimizationlogic.handle_exit_obj.is_user_running == False:
        print('Stopping the measurement!')
        # Update the pulse with the maximum number
        fomlogic.update_fom(10**10, status_code=-1)
        break
        
    #######################################################################################################
    # Get the Controls
    #######################################################################################################
    # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
    controlslogic.are_pulses_calculated = False
    # Get the controls from the controls logic
    pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
    #######################################################################################################
    # Perform the measurement
    #######################################################################################################
    
    # save the pulses as .txt files (predefined methods doesn't allow us to upload a numpy array as 
    # parameter)
    np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\amplitude.txt',np.column_stack((timegrids[0], pulses[0])))
    np.savetxt(r'C:\Users\Mesoscopic\Desktop\quocs_pulses\phase.txt',np.column_stack((timegrids[0], pulses[1])))
    time.sleep(0.2)
    
    # predefined method
    dictparamsall = pulsedmasterlogic.generate_method_params
    ############################ Uploading all predefined methods needed ###################################################
    # predefined method
    dict_qst = dictparamsall["sts_oc"]
    dict_qst["name"] = 'stsoc'
    dict_qst["length"] = length_oc
    dict_qst["filename_amplitude"] = filename_amplitude
    dict_qst["filename_phase"] = filename_phase
    dict_qst["folder_path"] = folder_path

    # generate the sequence
    pulsedmasterlogic.generate_predefined_sequence('sts_oc',kwarg_dict = dict_qst)  
    
    time.sleep(1)
    
    # upload the sequence
    pulsedmasterlogic.sample_ensemble(dict_qst["name"],True)
    while pulsedmasterlogic.status_dict['sampload_busy'] or pulsedmasterlogic.status_dict['sampling_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:     
        time.sleep(1)

    #print('Finished uploading', dictparams_oc["name"], '!')
         
    # make sure everything is finished (crucial)
    #time.sleep(1)#time.sleep(5)    
    
     # start the measurement
    #print('Starting the measurement!')
    pulsedmasterlogic.toggle_pulsed_measurement(True)
    time.sleep(1)
    
    # make sure the measurement started
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)

    measurement_start_time = time.time()
    
    while time.time() <= measurement_start_time + runtime:
        time.sleep(0.2)
        
        # option to stop the measurement
        if pulsedmasterlogic.globalrun == False:
            print('Stopping the measurement!')
            break

    # option to stop the measurement
    if pulsedmasterlogic.globalrun == False:
        print('Stopping the measurement!')
        pulsedmasterlogic.toggle_pulsed_measurement(False)
        break
        
    #time.sleep(1)
            
    # Stop the measurement
    pulsedmasterlogic.toggle_pulsed_measurement(False)
    
    # Make sure it stopped (Wait until the Picoscope card sends the data)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.1)
    
    #######################################################################################################
    # Analysis
    ######################################################################################################
    signal_data = pulsedmeasurementlogic.signal_data
    
    # calculate the fom
    up_norm = signal_data[1,0] / signal_data[1,0]
    down_norm = signal_data[1,1] / signal_data[1,0]
    
    fom = np.real(1 - (up_norm - down_norm))

    # save the fom to plot its evolution later
    fom_all.append(fom)
    
    # Update the figure of merit and the standard deviation to the fom logic
    fomlogic.update_fom(fom, 0.001,status_code=0)
    #fomlogic.update_fom(fom, std, status_code=0)
    
    # update the last time the fom is calculated
    last_time_fom = time.time()
    
# when did the optimization stop?
opt_end_time = time.time()

print('It took QuOCS ' + str(opt_end_time-opt_start_time) + ' s to optimize the pulse!')

print("Optimization finished")

## Save the results

In [ ]:
# Best controls
optimizer_obj = optimizationlogic.optimization_obj
pulses_list, time_grids_list, parameters_list = optimizer_obj.opt_alg_obj.get_best_controls()
t_amplitude = time_grids_list[0]
amplitude = pulses_list[0]
t_phase = time_grids_list[1]
phase = pulses_list[1]

In [ ]:
#np.savetxt(r'filepath\amplitude_200ns_12072022.txt',np.column_stack((t_amplitude, amplitude)))
#np.savetxt(r'filepath\phase_200ns_121072022.txt',np.column_stack((t_phase, phase)))
#np.savetxt(r'filepath\fom_all_200ns_12072022.txt',fom_all)